In [81]:
# coding: utf8

#Import

In [82]:
import os
import sys
import json
import pprint
import urllib3
import pandas as pd
from urllib3 import make_headers
from datetime import datetime, date

#Confinguration

In [83]:
pp = pprint.PrettyPrinter(indent=4)
http = urllib3.PoolManager()
make_headers(accept_encoding=True)

{'accept-encoding': 'gzip,deflate'}

In [84]:
HERE = os.getcwd() + '/'
index = 'lmd'
typeidx = 'project'
host = os.environ['LMD_ELASTIC_HOST']
PROJECTS_CSV = u'projects.csv' 

#Class Project

In [85]:
class Project():
    """
    """
    
    def __init__(self, name, desc='...', date=date.today(), link='#', icon='', team=''):
        self.name = name
        self.description = desc
        self.date = date
        self.link = link
        self.icon = icon
        self.team = team
        
        self.coworkers = []
        self.technologies = []
        
        self._id = 0
        
    def setId(self, _id):
        self._id = _id
        
    @property
    def getId(self):
        return self._id
    
    def addCoworkers(self, coworkers=[]):
        for f in coworkers: 
            if f not in self.coworkers: self.coworkers.append(f)

    def addTech(self, tech=[]):
        for t in tech: 
            if t not in self.technologies: self.technologies.append(t)
    
    @property
    def show(self):
        return {u'name': self.name,
           u'description': self.description,
           u'date': self.date.strftime(u'%Y-%m-%d'),
           u'link': self.link,
           u'icon': self.icon,
           u'team': self.team,
           u'coworkers': self.coworkers,
           u'technologies': self.technologies
          }
    
    @property
    def showstr(self):
        return json.dumps(self.show)

#Loader

### connection

In [86]:
res = http.request('GET', host)
pp.pprint(json.loads(res.data))

{   u'cluster_name': u'elasticsearch',
    u'name': u'Iron Man',
    u'tagline': u'You Know, for Search',
    u'version': {   u'build_hash': u'72cd1f1a3eee09505e036106146dc1949dc5dc87',
                    u'build_snapshot': False,
                    u'build_timestamp': u'2015-11-18T22:40:03Z',
                    u'lucene_version': u'5.3.1',
                    u'number': u'2.1.0'}}


In [87]:
df = pd.read_csv(HERE + PROJECTS_CSV, sep=';', lineterminator=u'\r')

In [88]:
df.head()

,id,name,description,date,link,icon,team,coworkers,technologies
0,_weshare,weshare,NaN,01/10/13,http://weshare-isepdev.rhcloud.com/,weshare.svg,G8B,1|9|6|7|8,PHP|JavaScript|MySQl
1,_flatracing,FlatRacing,NaN,01/03/14,http://flatracing.github.io,flatracing.svg,Div&Puib,1,JAVA (with JavaScript verison)
2,_isepapp,Isepapp,NaN,01/03/15,http://purple-isepdev.rhcloud.com/Isepapp,isepapp.svg,PurpleConsulting,2|5|4,JAVA|MySQl|jQuery|Bootstrap


In [89]:
for idx, p in df.iterrows():
    pro = Project(name = p[u'name'],
                  desc = u'...', 
                  date = datetime.strptime(p[u'date'], u'%d/%m/%y'),
                  link = p[u'link'],
                  icon = p[u'icon'], team = p[u'team'])
    pro.addCoworkers(p[u'coworkers'].split('|'))
    pro.addTech(p[u'technologies'].split('|'))
    #response = esConn.index(pro.show, 'lmd', 'project', p[u'id'])
    http.request('POST','/'.join([host, index, typeidx, p[u'id']]), body=pro.showstr)
    #pp.pprint(pro.show)

In [90]:
lmd = Project(name=u'This web site is also a project (',
                  desc=u'...', 
                  date=date(2015, 4, 1),
                  link=u'',
                  icon=u'dd.svg', team=u'Me')
lmd.addTech([u'Ember.js', u'Skeleton.css', u'Divshot'])

In [91]:
#curl -XDELETE http://localhost:9200/lmd/project/?pretty
#http.request('DELETE', )